In [ ]:
import os
import urllib.request
import re

import tiktoken
import torch

from torch.utils.data import Dataset, DataLoader


In [ ]:
if not os.path.exists('the-verdict.txt'):
    url = ('https://github.com/rasbt/LLMs-from-scratch/tree/main/ch02/01_main-chapter-code/the-verdict.txt')
    file_path = 'the-verdict.txt'
    urllib.request.urlretrieve(url,file_path)

## 2.2 Converting text to tokens

In [2]:
with open('the-verdict.txt','r',encoding='utf-8') as f:
    raw_text = f.read()

In [3]:
len(raw_text)

20479

In [4]:
result = re.split(r'([,.:;?_!"()|\']|--|\s)',raw_text)
result = [item.strip() for item in result if item.strip()]
preprocessed = result


In [5]:
len(preprocessed)

4690

In [6]:
preprocessed[:10]

['I',
 'HAD',
 'always',
 'thought',
 'Jack',
 'Gisburn',
 'rather',
 'a',
 'cheap',
 'genius']

## 2.3 Converting Tokens to token IDs

In [7]:
all_words = set(preprocessed)
vocab_size = len(all_words)

In [8]:
vocab = {token:integer for integer,token in enumerate(all_words)}

In [9]:
class SimpleTokenizerV1 :
    def __init__(self,vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self,text):
        preprocessed = re.split(r'([,.:;?_!"()|\']|--|\s)',text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self,ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])',r'\1',text)
        return text

In [10]:
tokenizer = SimpleTokenizerV1(vocab)

## 2.4 Adding special context tokens

In [11]:
text2 = "Hello. do you like tea. is this --a test?"
tokenizer.encode(text2)

KeyError: 'Hello'

In [12]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>","<|unk|>"])

vocab = {token:intger for intger,token in enumerate(all_tokens)}

In [13]:
len(vocab)

1132

In [16]:
class SimpleTokenizerV2 :
    def __init__(self,vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self,text):
        preprocessed = re.split(r'([,.:;?_!"()|\']|--|\s)',text)

        preprocessed = [item.strip() for item in preprocessed if item.strip()]

        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self,ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])',r'\1',text)
        return text

In [19]:
tokenizer = SimpleTokenizerV2(vocab)
text2 = "Hello, do you like tea. is this --a test?"
tokenizer.encode(text2)

[1131, 5, 355, 1126, 628, 975, 7, 584, 999, 6, 115, 1131, 10]

In [20]:
tokenizer.decode(tokenizer.encode(text2))

'<|unk|>, do you like tea. is this -- a <|unk|>?'

## 2.5 Byte pair encoding

In [22]:
tiktoken.__version__

'0.9.0'

In [23]:
tokenizer = tiktoken.get_encoding('gpt2')

In [24]:
tokenizer.encode("Hello World")

[15496, 2159]

In [25]:
tokenizer.decode(tokenizer.encode("Hello World"))

'Hello World'

In [28]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
    "of someknown place"
)
tokenizer.encode(text,allowed_special={"<|endoftext|>"})

[15496,
 11,
 466,
 345,
 588,
 8887,
 30,
 220,
 50256,
 554,
 262,
 4252,
 18250,
 8812,
 2114,
 1659,
 617,
 4002,
 1295]

## 2.6 Data Sampling with a sliding window

In [32]:
with open('the-verdict.txt','r',encoding='utf-8') as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [33]:
enc_sample = enc_text[50:]

In [37]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size]

print(f'x:{x}')
print(f'     y:{y}')

x:[290, 4920, 2241, 287]
     y:[4920, 2241, 287]


In [39]:
torch.__version__

'2.6.0+cpu'

In [ ]:
class GPTDatasetV1():
    def __init__(self,txt,tokenizer,max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt,allowed_special={"<|endoftext|>"})

        for i in range(0,len(token_ids)-max_length,stride):
            input_chunk = token_ids[i:i+max_length]
            output_chunk = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(output_chunk))
        
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self,idx):
        return self.input_ids[idx], self.target_ids[idx]

In [45]:
def create_data_loader_v1(txt,batch_size =2, max_length = 256,
                       stride = 128, shuffle= True, drop_last = True,
                       num_workers = 0):
    tokenizer = tiktoken.get_encoding('gpt2')
    dataset = GPTDatasetV1(txt ,tokenizer, max_length, stride)

    dataloader = DataLoader(dataset,
                            batch_size=batch_size,
                            shuffle = shuffle,
                            drop_last = drop_last,
                            num_workers = num_workers)
    
    return dataloader

In [43]:
with open('the-verdict.txt','r',encoding='utf-8') as f:
    raw_text = f.read()

In [54]:
dataloader = create_data_loader_v1(raw_text,batch_size=8,max_length = 4, stride = 4, shuffle=False)

data_iter  = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)


[tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]]), tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])]


In [55]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[  287,   262,  6001,   286],
        [  465, 13476,    11,   339],
        [  550,  5710,   465, 12036],
        [   11,  6405,   257,  5527],
        [27075,    11,   290,  4920],
        [ 2241,   287,   257,  4489],
        [   64,   319,   262, 34686],
        [41976,    13,   357, 10915]]), tensor([[  262,  6001,   286,   465],
        [13476,    11,   339,   550],
        [ 5710,   465, 12036,    11],
        [ 6405,   257,  5527, 27075],
        [   11,   290,  4920,  2241],
        [  287,   257,  4489,    64],
        [  319,   262, 34686, 41976],
        [   13,   357, 10915,   314]])]


## 2.7 Creating token embeddings

In [56]:
input_ids = torch.tensor([2,3,5,1])

In [57]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size,output_dim)

In [58]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


## 2.8 Encoding word positions

In [59]:
vocab_size = 50276
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size,output_dim)

In [60]:
max_length = 4
dataloader = create_data_loader_v1(raw_text,
                                   batch_size=8,
                                   max_length = 4, 
                                   stride = 4, 
                                   shuffle=False)

data_iter  = iter(dataloader)
inputs, targets = next(data_iter)


In [61]:
print("Token IDS:\n", inputs)

print("\nInput Shape:\n", inputs.shape)

Token IDS:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Input Shape:
 torch.Size([8, 4])


In [62]:
token_embeddings = token_embedding_layer(inputs)
token_embeddings.shape

torch.Size([8, 4, 256])

In [63]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [65]:
pos_embedding_layer = pos_embedding_layer(torch.arange(max_length))

In [68]:
pos_embedding_layer.shape

torch.Size([4, 256])

In [69]:
token_embeddings.shape

torch.Size([8, 4, 256])

In [71]:
input_embeddings = token_embeddings+token_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
